# Modelado semántico del vocabulario
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

El modelado basado en vocabularios o léxico tiene una gran cantidad de aplicaciones, las cuales son suelen explotar la gran cantidad de grados de libertad en el preprocesamiento, toquenizado y pesado, para ajustar a las tareas deseadas. Sin embargo, dicho modelado tiene problemas basados en la semántica:

- si las consultas tienen palabras no conocidas (fuera de vocabulario), no es posible ligar a palabras similares por semántica 
- las palabras similares en el vocabulario pero no iguales, no puede relacionarse para reducir un vocabulario, o para cualquier otro efecto
- las palabras que se escriben igual pero tienen identico significado serán puestas en la misma bolsa

De manera más detallada, se tendrán problemas al manejar sinónimos, hipónimos, antónimos, etc. Adicionalmente, los errores serán tratados como palabras diferentes y no ajustados al mejor calce conocido (a menos que exista un corrector ortográfico entre el preprocesamiento).

Estos problemas se buscan reducir con representaciones semánticas de los vocabularios y de los textos, esto sería equivalente. El intento sería buscar por conceptos más que por palabras que lo describan.
En este punto aparecen los modelos semánticos, los cuales suelen ser representaciones vectoriales densas _embeddings_, de alta dimensión pero mucho menor que las representaciones léxicas (i.e., pasarían a unos pocos cientos en lugar de varias decenas de miles o cientos de miles de coordinadas).

# Word embeddings
Las representaciones de vectores densos de las palabras se suelen construir utilizando la [_hipótesis distribucional_](https://aclweb.org/aclwiki/Distributional_Hypothesis), que se puede resumir que las palabras con contextos similares (palabras que la rodean) tienen un significado similar. Entonces, para obtener la semántica, es necesario comparar contextos; lo anterior se puede llevar a cabo de diferentes maneras.

- Latent semantic analysis (LSA): La matriz $W_{m,n}^*$  con pesos basados en frecuencias es usada para reducir el numero de filas (palabras) mediante _Singular Value Decomposition_ (SVD). La similitud adecuada entre vectores coseno en el espacio de dimensión reducida. Ver [@Dumais2004] para más información.
- Latent semantic indexing (LSI): Similar a LSA pero usando esquemas de pesado diferentes, adicionalmente se usa Trucated SVD (TSVD) en lugar de SVD ya que puede seleccionar componentes más importantes. Para más detalles ver [@Hofman1999].
- Word2Vec. Se construye una red neuronal con una matriz de pesos del tamaño del vocabulario por una dimensión seleccionada, a partir de los contextos. Revisa los textos de entrenamiento mediante una ventana móvil. De manera especial, se tiene que el centro de la ventana es la palabra objetivo y el contexto esta a su alrededor. Se intenta predecir mediante el contexto la palabra central, y se ajusta mediante propagación hacia atrás. La función de perdida suele ser [softmax](https://en.wikipedia.org/wiki/Softmax_function) o hierachical softmax. Los embeddings no son la capa de salida sino una capa atrás. Para más detalles ver [@MCCD2013].
- Glove. Se utiliza una matriz de coocurrencia de términos dentro de una ventana deslizante que analiza una colección de textos. La matriz es altamente dispersa. Se utiliza TSVD para bajar su dimensión. Los detalles se pueden consultar en [@PSM2014]. 
- FastText. Muy similar a Word2Vec pero añade manejo de palabras fuera de diccionario mediante subwords (similar a q-gramas de carácteres por cada palabra). Ver [@BGJM2017] para mayores detalles.

# Language models
En los word embeddings, cada palabra del vocabulario tiene asignado un vector denso de manera estática, fruto de su semántica basada en la hipótesis distribucional. Los modelos de lenguaje van más allá, intentando no solo tener en cuenta una palabra para el vector, si no que el vector mismo es dependiente del contexto, por lo cual puede desambiguar de manera natural palabras idénticas (homónimos) usando dicha información contextual. Adicionalmente, capturan información relevante de grandes corpus de texto, aportando muchas veces información de un mundo (i.e., si se entrenan usando la wikipedia, tendrán información relavante de multiples dominios).

La punta de lanza de los modelos de lenguaje es el aprendizaje profundo, y más precisamente, con el uso de _transformers_. Uno de los modelos de lenguaje más utilizado es BERT [@VSPU2017]. Estos modelos están fuera del alcance de nuestro curso pero se invitá al interesado a revisar más sobre ellos <https://huggingface.co/>.

## Nota
Este notebook esta basado en <https://github.com/sadit/SimilaritySearchDemos/blob/main/Glove/Glove.ipynb>. En el notebook original se muestran visualizaciones basadas en todos los vecinos cercanos (se verán más adelante en el curso).

# Ejemplo

El resto del documento muestra como usar word embeddings Glove para precalculados, para resolver tareas de vecinos cercanos y resolución de analogías.

In [1]:
using Pkg
Pkg.activate(".")
using SimilaritySearch, Plots, LinearAlgebra, Embeddings, HypertextLiteral

  Activating project at `~/IR-2022/Unidades`


In [2]:
function load_dataset()
    emb = load_embeddings(GloVe{:en}, 2)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end

    db = MatrixDatabase(emb.embeddings)
    db, emb.vocab
end

function create_index()
    db, vocab = load_dataset()
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index)
    optimize!(index, MinRecall(0.9))
    index, vocab
end


create_index (generic function with 1 method)

### El índice métrico se crea, además se toma el vocabulario

In [3]:
@time index, vocab = create_index()
vocab2id = Dict(w => i for (i, w) in enumerate(vocab));

 16.942226 seconds (11.69 M allocations: 3.329 GiB, 7.05% gc time, 17.64% compilation time)


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

In [4]:
let k = 32
    @time I, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    @time gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    macrorecall(gI, I)
end

 18.988572 seconds (4.40 M allocations: 3.757 GiB, 1.19% gc time, 6.21% compilation time)
245.906001 seconds (705.52 k allocations: 131.429 MiB, 0.15% compilation time)


0.94238015625

### Búsqueda y presentación de los resultados

In [5]:
function search_and_display(index, vocab, q, res, k, qword)
    res = reuse!(res, k)
    @time search(index, q, res)

    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$(vocab[id])</td><td>$id</td><td>$(round(d, digits=3))</td></tr>")
    end

    display(@htl """<h2>resultados for "$qword"</h2>
    <table>
    <th>  <td>word</td> <td>id</td> <td>dist</td> </th>
        $L
    </table>
    """)
end

function search_and_display(index, vocab, qid::Integer, res, k=maxlength(res))
    search_and_display(index, vocab, index[qid], res, k, vocab[qid])
end

display(@htl "<h1>Ejemplos de búsqueda (aleatorios)</h1>")
res = KnnResult(7)
for i in 1:3
    for qid in rand(1:length(vocab))
        search_and_display(index, vocab, qid, res)
    end
end

<h1>Ejemplos de búsqueda (aleatorios)</h1>

  0.001049 seconds (4 allocations: 1.844 KiB)


1,zetina,293562,-0.0
2,lebov,328833,0.241
3,herdahl,348355,0.259
4,funches,317751,0.265
5,dokur,231975,0.267
6,spiridonova,373226,0.267
7,nikc,350644,0.273


  0.001098 seconds (6 allocations: 7.594 KiB)


1,one-horse,394593,0.0
2,four-wheeler,394105,0.261
3,chauffeur-driven,399393,0.262
4,whistle-stop,381400,0.268
5,4-wheeled,396367,0.284
6,35-seat,322846,0.285
7,five-seater,397869,0.286


  0.000435 seconds (4 allocations: 1.844 KiB)


1,kornwestheim,325549,0.0
2,untertürkheim,365776,0.23
3,graben-neudorf,382978,0.28
4,renningen,389859,0.28
5,neckarelz,337321,0.293
6,thamshavn,355234,0.297
7,holzkirchen,379140,0.304


In [6]:
search_and_display(index, vocab, vocab2id["glove"], res, 15)

  0.000331 seconds (4 allocations: 1.844 KiB)


1,glove,10923,0.0
2,ball,1084,0.369
3,gloves,10808,0.369
4,plate,4365,0.379
5,pads,15978,0.418
6,infield,16731,0.435
7,bounced,9154,0.435
8,hat,5626,0.438
9,bat,4926,0.439
10,fielder,10241,0.447
11,pitch,3099,0.45


In [7]:
# Analogias

In [8]:
function analogy(a, b, c, k)
	v = index[vocab2id[a]] - index[vocab2id[b]] + index[vocab2id[c]]
	normalize!(v)
	search_and_display(index, vocab, v, res, k, "<$a> - <$b> + <$c>")
end

analogy("father", "man", "woman", 5)
analogy("fireman", "man", "woman", 5)
analogy("policeman", "man", "woman", 5)
analogy("mississippi", "usa", "france", 5)

  0.000107 seconds (2 allocations: 416 bytes)


1,mother,809,0.098
2,daughter,1132,0.132
3,wife,703,0.147
4,husband,1328,0.172
5,grandmother,7401,0.189


  0.000184 seconds (2 allocations: 1.438 KiB)


1,fireman,27345,0.156
2,firefighter,15812,0.303
3,paramedic,33841,0.393
4,rescuer,44915,0.439
5,passerby,53776,0.459


  0.000061 seconds (2 allocations: 416 bytes)


1,policeman,6857,0.144
2,wounding,6118,0.285
3,policemen,4984,0.295
4,passerby,53776,0.306
5,wounded,1392,0.331


  0.000279 seconds (2 allocations: 1.438 KiB)


1,france,388,0.468
2,rhine,13957,0.488
3,coast,955,0.506
4,brittany,15877,0.506
5,southern,483,0.506


# Actividades 
- Como se menciono, el uso de redes neuronales se vuelve muy popular para aprender la semántica desde el texto. Comente las razones justificando sus comentarios en términos de aprendizaje y requerimientos computacionales.
- Discuta que tipo de métrica se usa y porque.
- ¿Cómo se podrían usar los word-embeddings para búsqueda de documentos completos?
- Como ajustaría la métrica.
- Si usa Julia, revisé el paquete `Embeddings.jl`, `Word2Vec.jl`.
- Si usa Python, revisé los paquetes `fastText` y `word2vec`.
- Glove <https://nlp.stanford.edu/projects/glove/>
- Reproduzca el ejercicio de este notebook (no reproduzca la búsqueda exhaustiva, ya que le tomará muchísimo tiempo), use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Anote sus reflexiones.

# Bibliografía
- [Dumais2004] Dumais, S. T. (2004). Latent semantic analysis. Annu. Rev. Inf. Sci. Technol., 38(1), 188-230.
- [Hofmann1999] Hofmann, T. (1999, August). Probabilistic latent semantic indexing. In Proceedings of the 22nd annual international ACM SIGIR conference on Research and development in information retrieval (pp. 50-57).
- [MCCD2013]: Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781.
- [PSM2014] Pennington, J., Socher, R., & Manning, C. D. (2014, October). Glove: Global vectors for word representation. In Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP) (pp. 1532-1543).
- [BGJM2017]: Bojanowski, P., Grave, E., Joulin, A., & Mikolov, T. (2017). Enriching word vectors with subword information. Transactions of the association for computational linguistics, 5, 135-146.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.